## Task1

#### Model językowy

In [62]:


authors = ["orzeszkowej", "prusa", "sienkiewicza"]

punct_marks = ",.?!:'-()"

bigrams = {}


for author in authors:
    bigrams[author] = {}
    
    for line in open("../resources/dane_pozytywistyczne/korpus_"+author+".txt"):
        prev = "<BOS>"
        if not line.strip():
            continue
        for word in (line.lower() + " <EOS>").split():
            
            if word[-1] in punct_marks:
                new_word = word[-1]
                word = word[:-1]
            
            else:
                new_word = None
            
            if prev not in bigrams[author]:
                bigrams[author][prev] = {}
            
            if word not in bigrams[author][prev]:
                bigrams[author][prev][word] = 1
            else:
                bigrams[author][prev][word] += 1
            
            if new_word:
                if word not in bigrams[author]:
                    bigrams[author][word] = {}
                
                if new_word not in bigrams[author][word]:
                    bigrams[author][word][new_word] = 1
                else:
                    bigrams[author][word][new_word] += 1
                
                prev = new_word
            else:
                prev = word





In [63]:
# normalize
import math

best = 0

for author in bigrams:
    for prev in bigrams[author]:
        all_bigrams = math.log(sum(bigrams[author][prev].values()))
        for word, value in bigrams[author][prev].items():
            bigrams[author][prev][word] = math.log(value) - all_bigrams
        bigrams[author][prev]["#unigram#"] = all_bigrams
        best = max(best, all_bigrams)


In [83]:
import os

eps = -best*0.50 #strzelane tak, zeby bylo jak najlepiej - co ciekawe im blizej zera tym lepiej ocenia sienkiewicza kosztem innych


hits = {author:0 for author in authors}
size = {author:0 for author in authors}

for file_name in os.listdir('../resources/dane_pozytywistyczne/testy1/'):
    res = {}
    for author in authors:
        res[author] = 0 #p-stwo pustego to 1
        if author in file_name:
            size[author] += 1

    for line in open('../resources/dane_pozytywistyczne/testy1/' + file_name):
        for author in authors:
            prev = "<BOS>"
            if not line.strip():
                continue
            for word in (line.lower() + " <EOS>").split():

                if word[-1] in punct_marks:
                    new_word = word[-1]
                    word = word[:-1]

                else:
                    new_word = None

                if prev not in bigrams[author]:
                    res[author] += eps
                else:
                    res[author] += bigrams[author][prev].get(word, eps)

                if new_word:
                    if word not in bigrams[author]:
                        res[author] += eps
                    else:
                        res[author] += bigrams[author][word].get(new_word, eps)

                    prev = new_word
                else:
                    prev = word
    result = max(res.values())
    for author in authors:
        if result == res[author] and author in file_name:
            hits[author] += 1
            break

print("Dobre", sum(hits.values())/sum(size.values())*100, "%")
for author in authors:
    print("Dla {} trafione {}/{}".format(author, hits[author],size[author]))



Dobre 73.33333333333333 %
Dla orzeszkowej trafione 6/12
Dla prusa trafione 21/21
Dla sienkiewicza trafione 17/27
